In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing Modules

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

### Loading the dataset

In [ ]:
df=pd.read_csv('/kaggle/input/wine-quality/winequalityN.csv')
df.head()

In [ ]:
#stastical info

df.describe()

In [ ]:
# datatype info

df.info()

### Preprocessing the dataset

In [ ]:
#check for null values

df.isnull().sum()

In [ ]:
feature_with_na=[feature for feature in df.columns if df[feature].isnull().sum()>0]

In [ ]:
df[feature_with_na].head(10)

In [ ]:
for feature in feature_with_na:
    df[feature].fillna(df[feature].mean(), inplace=True)
df.isnull().sum()

### Exploratory Data Analysis

In [ ]:
#create box plots
fig,ax = plt.subplots(ncols=6,nrows=2,figsize=(20,10))
index=0
ax=ax.flatten()

for col, value in df.items():
    if col!='type':
        sns.boxplot(y=col,data=df,ax=ax[index])
        index+=1
    

In [ ]:
# create distplot
fig,ax = plt.subplots(ncols=6,nrows=2,figsize=(20,10))
index=0
ax=ax.flatten()

for col, value in df.items():
    if col!='type':
        sns.distplot(value,ax=ax[index])
        index+=1


In [ ]:
# log transformation
df['free sulfur dioxide']=np.log(1+df['free sulfur dioxide'])

In [ ]:
sns.distplot(df['free sulfur dioxide'])

In [ ]:
sns.countplot(df['type'])

In [ ]:
sns.countplot(df['quality'])

### Correlation Matrix

In [ ]:
corr=df.corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr,annot=True, cmap='coolwarm')

### Input Split

In [ ]:
X=df.drop(columns=['type','quality'])
y=df['quality']

### Class Imbalancement

In [ ]:
y.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE
oversample=SMOTE(k_neighbors=4)
#transform the dataset
X,y=oversample.fit_resample(X,y)

In [ ]:
y.value_counts()

### Model Training

In [ ]:
#classify function
from sklearn.model_selection import cross_val_score, train_test_split
def classify(model,X,y):
    X_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25, random_state=43)
    #train the model
    model.fit(X_train,y_train)
    print("Accuracy", model.score(x_test,y_test)*100)
    
    #cross-validation
    score=cross_val_score(model,X,y,cv=5)
    print("CV Score", np.mean(score)*100)

In [ ]:
from sklearn.linear_model import LogisticRegression
model= LogisticRegression()
classify(model,X,y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
classify(model,X,y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
classify(model,X,y)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model=ExtraTreesClassifier()
classify(model,X,y)